In [ ]:
import pybaseball
import pandas as pd
from pybaseball import statcast
pybaseball.cache.enable()

In [ ]:
# Calculate the correlation matrix
correlation_matrix = statcast.corr(numeric_only=True)

# Extract the correlations with 'estimated_woba'
woba_correlations = correlation_matrix['estimated_woba'].sort_values(ascending=False)

# Print the correlations
print("Correlations with estimated_woba:")
print(woba_correlations)

In [ ]:
# Convert the 'description' column to boolean: True if 'swinging_strike', False otherwise
statcast['swinging_strike'] = statcast['description'] == 'swinging_strike'

# Calculate the correlation matrix
correlation_matrix = statcast.corr(numeric_only=True)

# Extract the correlations with 'swinging_strike'
swinging_strike_correlations = correlation_matrix['swinging_strike'].sort_values(ascending=False)

# Print the correlations
print("Correlations with swinging_strike:")
print(swinging_strike_correlations)


In [ ]:
# Calculate the correlation matrix
correlation_matrix = statcast.corr(numeric_only=True)

# Extract the correlations with 'swinging_strike'
swinging_strike_correlations = correlation_matrix['swinging_strike'].sort_values(ascending=False)

# Print the correlations
print("Correlations with swinging_strike:")
print(swinging_strike_correlations)

In [ ]:
# Filter the DataFrame for pitches with woba above 0.325
high_woba_pitches = statcast[statcast['estimated_woba'] > 0.325].copy()

# Calculate the average of other stats for these pitches
average_stats = high_woba_pitches.mean(numeric_only=True)

# Display the average stats
print("Average stats for pitches with woba > 0.325:")
print(average_stats)


In [ ]:
# Filter the DataFrame for pitches with description 'swinging_strike'
swinging_strike_pitches = statcast[statcast['description'] == 'swinging_strike'].copy()

# Calculate the average of other stats for these pitches
average_stats_swinging_strike = swinging_strike_pitches.mean(numeric_only=True)


In [ ]:
# Group by horizontal and vertical position, then calculate the average WOBA, ignoring NaN values
average_woba_by_position = statcast.groupby(['horz_position_of_pitch', 'vertical_pos_of_pitch'])['estimated_woba'].mean().reset_index()

# Print the first few rows of the result
print(average_woba_by_position.head())

# Impute NaN values using a simple averaging approach
# neighborhood size is increased to .2
for index, row in average_woba_by_position.iterrows():
    if pd.isna(row['estimated_woba']):
        # Define a small neighborhood around the point
        neighborhood_size = 0.2  # Adjust as needed

        # Filter for points within the neighborhood
        nearby_points = average_woba_by_position[
            (average_woba_by_position['horz_position_of_pitch'] >= row['horz_position_of_pitch'] - neighborhood_size) &
            (average_woba_by_position['horz_position_of_pitch'] <= row['horz_position_of_pitch'] + neighborhood_size) &
            (average_woba_by_position['vertical_pos_of_pitch'] >= row['vertical_pos_of_pitch'] - neighborhood_size) &
            (average_woba_by_position['vertical_pos_of_pitch'] <= row['vertical_pos_of_pitch'] + neighborhood_size)
        ]['estimated_woba']

        # Calculate the mean of nearby WOBA values, ignoring NaN values
        nearby_woba_mean = nearby_points.mean(skipna=True)

        # If there are nearby points with WOBA values, fill the NaN with the mean
        if not pd.isna(nearby_woba_mean):
            average_woba_by_position.loc[index, 'estimated_woba'] = nearby_woba_mean

# After imputation, remove any remaining rows with NaN values
average_woba_by_position.dropna(subset=['estimated_woba'], inplace=True)